In [2]:
# =================================================================================
# Step 1: Install FEniCSx and other dependencies in Google Colab
# This section will install the necessary libraries if they are not already present.
# =================================================================================
import os
print("Installing system dependencies for gmsh...")
os.system("apt-get update && apt-get install -y libglu1-mesa")
print("System dependencies installed.")
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-release-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx
try:
    import dolfinx
except ImportError:
    print("Installing FEniCSx... This may take a few minutes.")
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx
    print("FEniCSx installation complete.")

try:
    import pyvista
except ImportError:
    print("Installing PyVista...")
    !pip install -q pyvista
    import pyvista
    print("PyVista installation complete.")

try:
    import gmsh
except ImportError:
    print("Installing gmsh...")
    !pip install -q gmsh
    import gmsh
    print("gmsh installation complete.")

import os
os.environ['PYVISTA_FORCE_STATIC_PLOTTING'] = 'true'
os.environ['GOOGLE_COLAB_BACKEND'] = 'ipython'


# =================================================================================
# Step 2: Import necessary libraries
# =================================================================================
import numpy as np
import ufl
from dolfinx import fem, mesh, plot
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
from dolfinx.fem.petsc import LinearProblem


# =================================================================================
# Step 3: Define the L-shaped geometry and create the mesh
# The domain is [-1, 1] x [-1, 1] with the top-right quadrant [0, 1] x [0, 1] removed.
# =================================================================================
# Define the communicator
comm = MPI.COMM_WORLD

# Create the L-shaped domain mesh using gmsh
try:
    from dolfinx.io import gmshio
    from gmsh import model
    import gmsh

    gmsh.initialize()
    if comm.rank == 0:
        # Define the points of the L-shaped polygon
        points = [
            gmsh.model.geo.addPoint(-1, -1, 0),
            gmsh.model.geo.addPoint(1, -1, 0),
            gmsh.model.geo.addPoint(1, 0, 0),
            gmsh.model.geo.addPoint(0, 0, 0),
            gmsh.model.geo.addPoint(0, 1, 0),
            gmsh.model.geo.addPoint(-1, 1, 0)
        ]
        # Define the lines connecting the points
        lines = [
            gmsh.model.geo.addLine(points[0], points[1]),
            gmsh.model.geo.addLine(points[1], points[2]),
            gmsh.model.geo.addLine(points[2], points[3]),
            gmsh.model.geo.addLine(points[3], points[4]),
            gmsh.model.geo.addLine(points[4], points[5]),
            gmsh.model.geo.addLine(points[5], points[0])
        ]
        # Create the curve loop and surface
        curve_loop = gmsh.model.geo.addCurveLoop(lines)
        surface = gmsh.model.geo.addPlaneSurface([curve_loop])
        # Synchronize the model
        gmsh.model.geo.synchronize()
        # Mark the physical surface
        gmsh.model.addPhysicalGroup(2, [surface], 1)
        # Generate the mesh
        gmsh.model.mesh.generate(2)

    # Convert the gmsh model to a dolfinx mesh
    domain, _, _ = gmshio.model_to_mesh(model, comm, 0, gdim=2)
    gmsh.finalize()

except ImportError:
    print("Gmsh is not available. Creating a fallback structured mesh.")
    # Fallback for environments where gmsh might not be easily available
    # This creates a structured mesh that is less ideal but works
    from dolfinx.mesh import create_rectangle, create_unit_square
    from dolfinx.mesh import meshtags

    # Create a square and then remove the unwanted part (conceptually)
    # The actual implementation here is to mesh the subdomains and combine
    # A simpler approach for a fallback is to use a unit square and transform
    # Note: This fallback does not generate the L-shape, it's a placeholder.
    # For a true L-shape, gmsh is highly recommended.
    domain = mesh.create_rectangle(comm, [np.array([-1, -1]), np.array([1, 1])], [32, 32], mesh.CellType.triangle)


# =================================================================================
# Step 4: Define the analytical solution and the source term `f`
# The PDE is ∇²u = -f. We derive `f` from the given analytical solution.
# u(x, y) = sin(πx) * cos(πy / 2)
# ∇²u = -π²sin(πx)cos(πy/2) - (π²/4)sin(πx)cos(πy/2) = -5/4 * π² * u(x,y)
# So, f = -∇²u = (5/4) * π² * sin(πx) * cos(πy / 2)
# =================================================================================
def analytical_solution(x):
    """
    Analytical solution u(x, y) = sin(πx) * cos(πy / 2).
    Args:
        x: A NumPy array of coordinates with shape [N, 3] (dolfinx provides 3D coords).
    Returns:
        The solution u at each coordinate.
    """
    x_ = x[:, 0]
    y_ = x[:, 1]
    return np.sin(np.pi * x_) * np.cos(np.pi * y_ / 2.0)

def source_term_f(x):
    """
    Computes the source term f = -∇²u.
    """
    x_ = x[:, 0]
    y_ = x[:, 1]
    return (5.0 * np.pi**2 / 4.0) * np.sin(np.pi * x_) * np.cos(np.pi * y_ / 2.0)

# =================================================================================
# Step 5: Set up the Finite Element problem
# =================================================================================
# Define the function space (P1 Lagrange elements)
V = fem.functionspace(domain, ("Lagrange", 1))

# Define the source term `f` as a dolfinx Function
f = fem.Function(V)
f.interpolate(source_term_f)

# Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Define the variational problem (bilinear and linear forms)
# a(u, v) = ∫ ∇u ⋅ ∇v dx
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
# L(v) = ∫ f * v dx
L = f * v * ufl.dx

# =================================================================================
# Step 6: Define and apply the Dirichlet boundary condition
# The boundary condition is u = u_D on the entire boundary,
# where u_D is the analytical solution.
# =================================================================================
# Interpolate the analytical solution into a function for the boundary condition
u_D = fem.Function(V)
u_D.interpolate(analytical_solution)

# Locate the boundary degrees of freedom
# `locate_entities_boundary` finds all facets on the exterior boundary
boundary_facets = mesh.locate_entities_boundary(
    domain, domain.topology.dim - 1, lambda x: np.full(x.shape[1], True)
)
boundary_dofs = fem.locate_dofs_topological(V, domain.topology.dim - 1, boundary_facets)

# Create the Dirichlet Boundary Condition
bc = fem.dirichletbc(u_D, boundary_dofs)


# =================================================================================
# Step 7: Solve the linear system
# =================================================================================
# Define the linear problem
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

print("Solving the linear system...")
uh = problem.solve()
print("Solution complete.")


# =================================================================================
# Step 8: Validate the solution by computing the L2 error
# =================================================================================
# Compute the L2 error ||u - u_h||_L2
error_form = fem.form((uh - u_D)**2 * ufl.dx)
error_local = fem.assemble_scalar(error_form)
error_L2 = np.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

# Compute the L2 norm of the analytical solution for relative error
norm_u_form = fem.form(u_D**2 * ufl.dx)
norm_u_local = fem.assemble_scalar(norm_u_form)
norm_u = np.sqrt(domain.comm.allreduce(norm_u_local, op=MPI.SUM))

print("\n--- Validation Results ---")
print(f"L2 Error (absolute): {error_L2:.4e}")
print(f"L2 Norm of analytical solution: {norm_u:.4e}")
print(f"Relative L2 Error: {error_L2 / norm_u:.4e}")
print("--------------------------\n")

# =================================================================================
# Step 9: Visualize the solution using PyVista
# =================================================================================
print("Visualizing the solution...")

# Create a PyVista plotter
plotter = pyvista.Plotter(window_size=[800, 800])

# Create a topology and geometry array for PyVista using the new function
topology, cell_types, geometry = plot.vtk_mesh(V)

# Create the PyVista grid object
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# Attach the solution values to the grid
grid.point_data["u"] = uh.x.array.real
grid.set_active_scalars("u")

# Add the grid to the plotter and configure the plot
actor = plotter.add_mesh(grid, show_edges=True, style='surface')
plotter.view_xy()
plotter.add_scalar_bar(title="Numerical Solution u_h", vertical=True)
plotter.camera.zoom(1.2)

# Show the plot
# In Colab, this will output a static image.
# In a local environment, it will be interactive.
pyvista.set_jupyter_backend("static")
plotter.show()

print("\nVisualization generated. If running in Colab, an image should appear above.")

Installing system dependencies for gmsh...
System dependencies installed.
Solving the linear system...
Solution complete.

--- Validation Results ---
L2 Error (absolute): nan
L2 Norm of analytical solution: nan
Relative L2 Error: nan
--------------------------

Visualizing the solution...

Visualization generated. If running in Colab, an image should appear above.
